In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import keras

from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation

In [ ]:
with np.load('data/train_and_test.npz') as data:
    X_test = data['X_test']
    y_train = data['y_train']
    X_train = data['X_train']

In [ ]:
plt.imshow(X_train[20000])
plt.title(y_train[20000])

## Different tries at various models

In [ ]:
labels = set(y_train)
img_size = 32
n_filters = 3
n_dense = 25

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(filters=32,
               kernel_size=(3,3), # filter size
               strides=(1,1),
               padding='same',
               input_shape=(32, 32, 3),
               activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))

cnn.add(Dropout(0.4))

cnn.add(Conv2D(filters=64,
               kernel_size=(2,2),
               strides=(1,1),
               padding='valid'))
cnn.add(Activation('relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))

cnn.add(Flatten())
cnn.add(Dense(64))
cnn.add(Activation('relu'))

cnn.add(Dropout(0.4))

cnn.add(Dense(1))
cnn.add(Activation('sigmoid'))

cnn.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
# https://becominghuman.ai/building-an-image-classifier-using-deep-learning-in-python-totally-from-a-beginners-perspective-be8dbaf22dd8

cnn = Sequential()
cnn.add(Conv2D(filters=32,
               kernel_size=(3,3), # filter size
               strides=(1,1),
               padding='same',
               input_shape=(32, 32, 3),
               activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))
cnn.add(Flatten())
cnn.add(Dense(units=128, activation='relu')) # hidden layer. TODO: optimize units value
cnn.add(Dense(units=1, activation='sigmoid'))
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Accuracy = 0.046 vv

In [ ]:
# https://www.dlology.com/blog/one-simple-trick-to-train-keras-model-faster-with-batch-normalization/
# https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

# bad model
cnn = Sequential()
cnn.add(Conv2D(32, (3,3), use_bias=False))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))

cnn.add(Conv2D(32, (3,3), use_bias=False))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

cnn.add(MaxPooling2D(pool_size=(2,2),
                     strides=2))

cnn.add(Conv2D(32, (3,3), use_bias=False))
cnn.add(BatchNormalization())
cnn.add(Activation('relu'))

cnn.add(Flatten())

cnn.add(Dense(units=1, activation='relu'))

cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Actually working model??? !! vv
#### Accuracy caps off at 15 epochs

In [ ]:
# https://www.kaggle.com/kentaroyoshioka47/cnn-with-batchnormalization-in-keras-94

img_rows, img_cols = 32, 32
input_shape = (img_rows, img_cols, 3)
batch_size = 128
num_classes = 10
epochs = 100
filter_pixel = 2
noise = 1
droprate = 0.25

model = Sequential()
#convolution 1st layer
model.add(Conv2D(32, kernel_size=(filter_pixel, filter_pixel), padding="same",
                 activation='relu',
                 input_shape=input_shape)) 
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2),
                       strides=2))
model.add(Dropout(droprate))

#convolution 2nd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',border_mode="same"))#1
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

#convolution 3rd layer
model.add(Conv2D(64, kernel_size=(filter_pixel, filter_pixel), activation='relu',border_mode="same"))#1
model.add(BatchNormalization())
model.add(MaxPooling2D())
model.add(Dropout(droprate))

#Fully connected 1st layer
model.add(Flatten()) #7
model.add(Dense(64,use_bias=False)) 
model.add(BatchNormalization())
model.add(Activation('relu')) 
model.add(Dropout(droprate))      

#Fully connected final layer
model.add(Dense(43)) 
model.add(Activation('softmax')) 

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(learning_rate=0.001),
              metrics=['accuracy'])

model.summary()

In [ ]:
EPOCHS = 15
BATCH_SIZE = 32
y_binary = to_categorical(y_train)
hist = model.fit(X_train, 
               y_binary, 
               epochs=EPOCHS,
               batch_size=BATCH_SIZE)

In [ ]:
result = model.predict(X_train)

In [ ]:
test_result = model.predict(X_test)

yay = [list(t).index(max(t)) for t in test_result]

with open('submission.csv', 'w') as file:
    file.write('id, labels\n')
    for i, label in enumerate(yay):
        file.write('{},{}\n'.format(i, label))